In [40]:
####Import Modules####
import tensorflow as tf
import random
import numpy as np
import math
import pandas as pd

####Set Random Seed ####
tf.set_random_seed(777) 

def cal_var(variances, per):
    all_cnt = len(variances)
    per = 100-per
    per_idx = int(all_cnt*(per/100))
    print(variances[per_idx])
    return variances[per_idx]
    
#def random_five_fold(data, num, indexs):
#    test_set = data[:, indexs[:2000]]
#    train_set = data[ :, indexs[2000:]]
#    #train_set = np.concatenate((train_set,data[(num+1)*2000:] ), axis=0)
#    return train_set , test_set

def top_of_variance(per, data_x):
    ##data_x['variance']
    ##calculate value  
    idx_list = np.where(data_x[:,-1]> per)
    ##return index 
    return idx_list

def set_train_three_layer(num,repeat, nodes, learning_rate):
    tf.reset_default_graph()
    train_a = 0
    test_a = 0
    X = tf.placeholder(tf.float32, [None, cnt_train])
    Y = tf.placeholder(tf.float32, [None, 2])

    W1 = tf.get_variable( shape= [cnt_train, nodes[0]], name='weight1' , initializer=tf.contrib.layers.xavier_initializer())
    b1 = tf.Variable(tf.random_normal([nodes[0]]), name='bias1')
    layer1 = tf.nn.relu(tf.matmul(X, W1) + b1)


    W2 = tf.get_variable(shape =[nodes[0], nodes[1]], name='weight2', initializer=tf.contrib.layers.xavier_initializer())
    b2 = tf.Variable(tf.random_normal([nodes[1]]), name='bias2')
    layer2 = tf.nn.relu(tf.matmul(layer1, W2) + b2)

    W3 = tf.get_variable(shape= [nodes[1], nodes[2]], name='weight3',initializer=tf.contrib.layers.xavier_initializer())
    b3 = tf.Variable(tf.random_normal([nodes[2]]), name='bias3')
    layer3 = tf.nn.relu(tf.matmul(layer2, W3) + b3)

    W4 = tf.get_variable(shape=[nodes[2], 2], name='weight4',initializer=tf.contrib.layers.xavier_initializer())
    b4 = tf.Variable(tf.random_normal([2]), name='bias4')
    hypothesis = tf.sigmoid(tf.matmul(layer3, W4) + b4)
    


    # cost/loss function
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=hypothesis, labels=Y))
    train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)



    # Accuracy computation
    # True if hypothesis>0.5 else False

    predicted = tf.argmax(hypothesis,1)
    correct_prediction = tf.equal(predicted,tf.argmax(Y,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, dtype=tf.float32))
    with tf.Session() as sess:
        # Initialize TensorFlow variables
        sess.run(tf.global_variables_initializer())

        for step in range(repeat):
            sess.run(train, feed_dict={X: train_x, Y: train_y})
            if step == repeat-1:
                ####Train Accuracy report####
                h, c, train_a = sess.run([hypothesis, predicted, accuracy],feed_dict={X: train_x, Y: train_y  })
                print("\nTrain Accuracy: ", train_a)
            if step % 20 == 0 :
                h,c, p,train_a = sess.run([hypothesis, cost ,predicted, accuracy],feed_dict={X: train_x, Y: train_y })
                print("\nCurrent Accuracy : ", train_a , "cost : ", c , "Current Step : ", step)
                if train_a > 0.95 :
                    break
        ######Accuracy Report#####
        h, c, test_a = sess.run([hypothesis, predicted, accuracy],feed_dict={X: test_x, Y: test_y})    
        print("\nTest Accuracy: ", test_a)
    
    return train_a, test_a
        
def set_train_four_layer(num ,repeat, nodes, learning_rate):
    tf.reset_default_graph()
    train_a = 0
    test_a = 0
    X = tf.placeholder(tf.float32, [None, cnt_train])
    Y = tf.placeholder(tf.float32, [None, 2])
    
    W1 = tf.get_variable( shape= [cnt_train, nodes[0]], name='Weight1' , initializer=tf.contrib.layers.xavier_initializer())
    b1 = tf.Variable(tf.random_normal([nodes[0]]), name='Bias1')
    layer1 = tf.nn.relu(tf.matmul(X, W1) + b1)
    
    W2 = tf.get_variable(shape =[nodes[0], nodes[1]], name='Weight2', initializer=tf.contrib.layers.xavier_initializer())
    b2 = tf.Variable(tf.random_normal([nodes[1]]), name='Bias2')
    layer2 = tf.nn.relu(tf.matmul(layer1, W2) + b2)
    
    W3 = tf.get_variable(shape= [nodes[1], nodes[2]], name='Weight3',initializer=tf.contrib.layers.xavier_initializer())
    b3 = tf.Variable(tf.random_normal([nodes[2]]), name='Bias3')
    layer3 = tf.nn.relu(tf.matmul(layer2, W3) + b3)

    W4 = tf.get_variable(shape = [nodes[2], nodes[3]] , name='Weight4' , initializer=tf.contrib.layers.xavier_initializer())
    b4 = tf.Variable(tf.random_normal([nodes[3]]), name='Bias4')
    layer4 = tf.nn.relu(tf.matmul(layer3, W4) + b4)

    W5 = tf.get_variable(shape = [nodes[3], 2],name='Weight5',initializer=tf.contrib.layers.xavier_initializer())
    b5 = tf.Variable(tf.random_normal([2]), name='Bias5')
    hypothesis = tf.matmul(layer4, W5) + b5

    # cost/loss function
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=hypothesis, labels=Y))
    train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)



    # Accuracy computation


    predicted = tf.argmax(hypothesis,1)
    correct_prediction = tf.equal(predicted,tf.argmax(Y,1))


    accuracy = tf.reduce_mean(tf.cast(correct_prediction, dtype=tf.float32))
    with tf.Session() as sess:
        # Initialize TensorFlow variables
        sess.run(tf.global_variables_initializer())

        for step in range(repeat):
            sess.run(train, feed_dict={X: train_x, Y: train_y})
            if step == repeat-1:
                ####Train Accuracy report####
                h, c, train_a = sess.run([hypothesis, predicted, accuracy],feed_dict={X: train_x, Y: train_y})
                print("\nTrain Accuracy: ", train_a)
            if step % 20 == 0 : 
                h, c, p,train_a = sess.run([hypothesis, cost ,predicted, accuracy],feed_dict={X: train_x, Y: train_y })
                print("\nCurrent Accuracy : ", train_a , "Cost : ",c , "Current Step : ", step)
                if train_a > 0.95 :
                    break

        ######Accuracy Report#####
        h, c, test_a = sess.run([hypothesis, predicted, accuracy],feed_dict={X: test_x, Y: test_y})    
        print("\nTest Accuracy: ", test_a)
    
    return train_a, test_a

In [68]:
####Read data####
#x_filename = input("Insert X dataset directory and name  : ")
#y_filename = input("Insert Y dataset directory and name : ")
x_filename = '/Users/taewan/Desktop/Ahn/DNN10000.csv'
xdata = pd.read_csv(x_filename)
ydata = np.genfromtxt('/Users/taewan/Desktop/CancerResult10000.csv', delimiter=",")
#conf_filename = input("Insert configure file directory and name : ")
xdata = xdata.as_matrix()
xdata = xdata[1:, 3:]   #variance 포함

/Users/taewan/.pyenv/versions/anaconda3-4.4.0/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [70]:
varis = xdata[:,-1]
print(varis)
print(xdata[:,-1])
varis.sort()
print(varis)

[48.24 145.976 306.64099999999996 ..., 14.790999999999999 36.227 16.209]
[48.24 145.976 306.64099999999996 ..., 14.790999999999999 36.227 16.209]
[6.82 7.736000000000001 7.792999999999999 ..., 2033.067 2084.964 2123.701]


In [19]:
conf_filename = '/Users/taewan/Desktop/Ahn/Git/BioDataLab/Data/input/relu_test_ps.csv'
conf = pd.read_csv(conf_filename)
print(conf)
train_accs = []
test_accs = []

[7943, 3393, 1525, 5854, 6271, 1762, 1060, 654, 6494, 5268, 1709, 9293, 3540, 3151, 5274, 86, 7897, 1048, 8638, 1438, 1910, 1163, 7181, 4686, 2165, 9872, 2240, 1565, 3604, 588, 9635, 7390, 4843, 6878, 7555, 1264, 8749, 7786, 8849, 9043, 8882, 5031, 8990, 90, 7180, 4401, 1207, 5255, 587, 7642, 3837, 5555, 2521, 258, 2926, 8992, 6195, 6865, 5774, 1402, 5677, 13, 5973, 5169, 7522, 7852, 4577, 5824, 5469, 2717, 676, 1012, 4419, 3469, 8272, 692, 4948, 9254, 3887, 9752, 2476, 9715, 6711, 5114, 1959, 2922, 3023, 6719, 7271, 5909, 8432, 5120, 9491, 4822, 3768, 6337, 285, 3374, 3543, 9705, 2070, 554, 3940, 5931, 2377, 9688, 5891, 5444, 8480, 6331, 2743, 4051, 9172, 8701, 7976, 5082, 6947, 628, 4781, 5988, 1701, 4809, 8950, 2610, 7017, 2692, 5162, 7949, 7691, 7570, 4497, 9306, 6047, 4183, 8557, 5478, 1530, 9595, 2426, 4955, 1656, 1477, 1475, 4990, 8587, 9835, 4130, 3612, 6348, 4880, 8473, 9396, 7447, 1955, 450, 7602, 359, 1767, 4189, 8827, 7554, 5734, 9550, 7765, 9155, 8191, 396, 2731, 9586, 207

[[-0.081 -0.223 -0.34299999999999997 ..., -0.34 -0.381 48.24]
 [0.37200000000000005 -0.021 0.037000000000000005 ..., -0.20800000000000002
  0.019 145.976]
 [1.09 0.14800000000000002 1.119 ..., 0.027000000000000003 1.192
  306.64099999999996]
 ..., 
 [0.061 -0.23800000000000002 -0.431 ..., -0.431 -0.45399999999999996
  14.790999999999999]
 [-0.1 0.127 -0.376 ..., -0.322 -0.446 36.227]
 [-0.348 -0.221 -0.43200000000000005 ..., -0.48100000000000004 -0.524
  16.209]]


In [62]:
indexs = list(range(len(xdata[1,:-1])))
random.shuffle(indexs)
# print(indexs)
variance_set = np.column_stack((xdata[:,indexs[2000:]] , xdata[:,-1]))
# print(variance_set)
variances = xdata[-1]
variances = np.sort(variances)
print(variances)

[-4.16 -3.819 -3.815 ..., 518.616 756.0319999999999 895.0989999999999]


In [61]:

for i in range(len(conf)):

    repeat, layer, node , learning_rate, gene = conf.iloc[i]
    nodes = list(map(int , node.split(" ")))

    j = 0
    cnt_train = len(xdata[:,-1])    
    
    # print(variances)
    idx = top_of_variance(cal_var(variances ,0.1) ,variance_set)
    variance_set = variance_set[idx]
    data_x = xdata[idx]
    data_x = data_x[:,:-1]   
#     variance_set = pd.concat([xdata.iloc[indexs[2000:]] , xdata.iloc[-1]])
    ###############################Edit############################
#     variance_set = pd.concat([xdata.iloc[:2000*j], xdata.iloc[2000*(j+1):]])

#     #print(variance_set.iloc[:,-1])
#     variances = variance_set.iloc[:,-1]
#     #print(variances)
#     variances = variances.as_matrix()
#     #print(variances)
#     variances = np.sort(variances)
#     #print(variances)
#     idx = top_of_variance(cal_var(variances, gene) , variance_set)
#     ###############################Edit############################
    
#     data_x = xdata.loc[idx]
#     data_x = data_x.as_matrix()
#     data_x = data_x[1:, 3:-1]
    data_y = ydata[1:, 1:]    # eliminate heading, string data
    # One-Hot-Encoding
    data_y = data_y.flatten()
    data_y = pd.get_dummies(data_y)


    
    ###############################Edit############################
    print(data_x)
    test_x = data_x[:,indexs[:2000]]
    train_x = data_x[:,indexs[2000:]]
    train_x = train_x.transpose()
    test_x = test_x.transpose()
#     print(data_y)

    train_y = data_y.loc[indexs[2000:],:]
    test_y = data_y.loc[indexs[:2000],:]
#     train_y, test_y = random_five_fold(data_y,j,indexs)
    ###############################Edit############################

    #print(train_y)
    if(conf.iloc[i]['layer'] == 3):
        train_acc , test_acc = (set_train_three_layer(i,repeat, nodes, learning_rate))
    elif(conf.iloc[i]['layer']== 4):
        train_acc , test_acc = (set_train_four_layer(i,repeat, nodes, learning_rate))
    train_accs.append(train_acc)
    test_accs.append(test_acc)


train_accs = pd.DataFrame(data=train_accs , 
                          index = list(range(len(conf))) , 
                          columns = ["train_acc"])
test_accs = pd.DataFrame(data=test_accs , 
                          index = list(range(len(conf))) , 
                          columns = ["test_acc"])

accuracies = pd.concat([train_accs, test_accs], axis=1)
conf = pd.concat([conf, accuracies] , axis = 1)
conf.to_csv('/home/tjahn/Git/Data/output/'+ conf_filename[:-4] +'.csv' , sep= ',')


206.9
[[0.575 1.2 1.163 ..., 0.408 0.23199999999999998 0.462]
 [0.545 1.004 1.0390000000000001 ..., 4.945 5.156000000000001 5.124]
 [0.743 0.963 0.909 ..., 1.694 1.381 1.527]
 ..., 
 [-2.016 -1.087 -2.638 ..., 0.43799999999999994 0.486 0.368]
 [1.845 1.892 1.78 ..., 0.8859999999999999 1.139 0.807]
 [0.905 1.139 1.1079999999999999 ..., -0.408 -0.419 -0.397]]


ValueError: Cannot feed value of shape (8012, 733) for Tensor 'Placeholder:0', which has shape '(?, 4521)'